In [1]:
import read_data_utils as utils
import regression_weight_tuning_EU_LG_UA as EU_LG_UA
import reorganization as reorg

In [2]:
import sys
import copy
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [3]:
file = 'train_all_0.csv'
dataloaders, dataset_sizes = utils.read_data(file, batch_size=8, mode='mse')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
model = nn.Sequential(
      nn.Linear(12,128),
      nn.ReLU(),
      nn.Linear(128,1),
      nn.ReLU()).to(device)

# model.load_state_dict(torch.load('/content/drive/MyDrive/新型學習演算法/hw3/acceptable_weights/a_regression.pt'))

criterion = nn.MSELoss()

optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

train_result = EU_LG_UA.train_model(model, criterion, optimizer, 
                  dataloaders, dataset_sizes, device,
                  PATH = '../weights/train_checkpoint.pt', 
                  epsilon = 1e-8,
                  num_epochs=50, n=10)

if train_result['result']:
  torch.save(model.state_dict(), '../acceptable_weights/before_reorg.pt')
  reorg_result, p = reorg.reorg_model(model, criterion, optimizer, dataloaders, dataset_sizes, device, lr_epsilon=1e-8 , n=10, rs=0.001)

torch.save(result, f'../acceptable_weights/after_reorganized_{p}.pt')